In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [2]:
disneyland_path = "Resources/DisneylandReviews.csv"
cacao_path = "Resources/flavors_of_cacao.csv"

In [3]:
disney_df = pd.read_csv(disneyland_path)
cacao_df = pd.read_csv(cacao_path)

In [4]:
disney_df

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong
...,...,...,...,...,...,...
42651,1765031,5,missing,United Kingdom,i went to disneyland paris in july 03 and thou...,Disneyland_Paris
42652,1659553,5,missing,Canada,2 adults and 1 child of 11 visited Disneyland ...,Disneyland_Paris
42653,1645894,5,missing,South Africa,My eleven year old daughter and myself went to...,Disneyland_Paris
42654,1618637,4,missing,United States,"This hotel, part of the Disneyland Paris compl...",Disneyland_Paris


In [5]:
cacao_df

,Company \n(Maker-if known),Specific Bean Origin\nor Bar Name,REF,Review\nDate,Cocoa\nPercent,Company\nLocation,Rating,Bean\nType,Broad Bean\nOrigin
0,A. Morin,Agua Grande,1876,2016,63%,France,3.75,,Sao Tome
1,A. Morin,Kpime,1676,2015,70%,France,2.75,,Togo
2,A. Morin,Atsane,1676,2015,70%,France,3.00,,Togo
3,A. Morin,Akata,1680,2015,70%,France,3.50,,Togo
4,A. Morin,Quilla,1704,2015,70%,France,3.50,,Peru
...,...,...,...,...,...,...,...,...,...
1790,Zotter,Peru,647,2011,70%,Austria,3.75,,Peru
1791,Zotter,Congo,749,2011,65%,Austria,3.00,Forastero,Congo
1792,Zotter,Kerala State,749,2011,65%,Austria,3.50,Forastero,India
1793,Zotter,Kerala State,781,2011,62%,Austria,3.25,,India


In [6]:
new_disney_df = disney_df[['Branch', 'Rating', 'Reviewer_Location', 'Year_Month']].copy()
new_disney_df = new_disney_df.rename(columns={"Branch":"branch","Rating":"rating","Reviewer_Location":"location","Year_Month":"year_month"})


In [7]:
#cacao_df.columns = [x.replace("\n", " ") for x in cacao_df.columns.to_list()]
cacao_df = cacao_df.rename(columns={cacao_df.columns[0]: "company", cacao_df.columns[3]:"year", cacao_df.columns[5]:"location", cacao_df.columns[6]:"cacao_rating", cacao_df.columns[8]: "bean_origin"})
# new_cacao_df = cacao_df[['Company \n(Maker-if known)', 'Company\nLocation', 'Rating', 'Broad Bean\nOrigin']].copy()
cacao_df

,company,Specific Bean Origin\nor Bar Name,REF,year,Cocoa\nPercent,location,cacao_rating,Bean\nType,bean_origin
0,A. Morin,Agua Grande,1876,2016,63%,France,3.75,,Sao Tome
1,A. Morin,Kpime,1676,2015,70%,France,2.75,,Togo
2,A. Morin,Atsane,1676,2015,70%,France,3.00,,Togo
3,A. Morin,Akata,1680,2015,70%,France,3.50,,Togo
4,A. Morin,Quilla,1704,2015,70%,France,3.50,,Peru
...,...,...,...,...,...,...,...,...,...
1790,Zotter,Peru,647,2011,70%,Austria,3.75,,Peru
1791,Zotter,Congo,749,2011,65%,Austria,3.00,Forastero,Congo
1792,Zotter,Kerala State,749,2011,65%,Austria,3.50,Forastero,India
1793,Zotter,Kerala State,781,2011,62%,Austria,3.25,,India


In [8]:
new_cacao_df = cacao_df[['company', 'location', 'cacao_rating', 'bean_origin','year']].copy()



In [9]:
new_cacao_df_2 = new_cacao_df.replace(['U.S.A.'], 'United States')

In [10]:
cacao_us_df = new_cacao_df_2.loc[new_cacao_df_2['location'] == 'United States']
cacao_us_df

,company,location,cacao_rating,bean_origin,year
23,Acalli,United States,3.75,Peru,2015
24,Acalli,United States,3.75,Peru,2015
44,Altus aka Cao Artisan,United States,2.75,Nicaragua,2016
45,Altus aka Cao Artisan,United States,3.00,Peru,2016
46,Altus aka Cao Artisan,United States,2.50,Bolivia,2016
...,...,...,...,...,...
1762,Zak's,United States,3.25,Domincan Republic,2015
1763,Zak's,United States,3.25,Madagascar,2015
1764,Zak's,United States,3.25,Peru,2015
1765,Zak's,United States,3.50,Belize,2015


In [11]:
#new_disney_df_cleaned = new_disney_df[['branch', 'rating', 'location', 'year']].copy()
new_disney_df_us = new_disney_df.loc[new_disney_df['location'] == 'United States']
new_disney_df_us

,branch,rating,location,year_month
10,Disneyland_HongKong,5,United States,2019-4
21,Disneyland_HongKong,3,United States,2019-4
22,Disneyland_HongKong,3,United States,2019-4
40,Disneyland_HongKong,3,United States,2019-4
43,Disneyland_HongKong,4,United States,2019-2
...,...,...,...,...
42635,Disneyland_Paris,1,United States,missing
42639,Disneyland_Paris,2,United States,missing
42647,Disneyland_Paris,2,United States,missing
42648,Disneyland_Paris,5,United States,missing


In [12]:
new_disney_df_us = new_disney_df_us[new_disney_df_us.year_month != 'missing']
new_disney_df_us

,branch,rating,location,year_month
10,Disneyland_HongKong,5,United States,2019-4
21,Disneyland_HongKong,3,United States,2019-4
22,Disneyland_HongKong,3,United States,2019-4
40,Disneyland_HongKong,3,United States,2019-4
43,Disneyland_HongKong,4,United States,2019-2
...,...,...,...,...
42082,Disneyland_Paris,5,United States,2011-2
42093,Disneyland_Paris,5,United States,2010-6
42100,Disneyland_Paris,1,United States,2011-1
42104,Disneyland_Paris,1,United States,2010-5


In [13]:
new_disney_df_us[['year','month']] = new_disney_df_us['year_month'].str.split('-', expand=True)
new_disney_df_us

C:\Users\ellio\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,branch,rating,location,year_month,year,month
10,Disneyland_HongKong,5,United States,2019-4,2019,4
21,Disneyland_HongKong,3,United States,2019-4,2019,4
22,Disneyland_HongKong,3,United States,2019-4,2019,4
40,Disneyland_HongKong,3,United States,2019-4,2019,4
43,Disneyland_HongKong,4,United States,2019-2,2019,2
...,...,...,...,...,...,...
42082,Disneyland_Paris,5,United States,2011-2,2011,2
42093,Disneyland_Paris,5,United States,2010-6,2010,6
42100,Disneyland_Paris,1,United States,2011-1,2011,1
42104,Disneyland_Paris,1,United States,2010-5,2010,5


In [14]:
new_disney_df_us_cleaned = new_disney_df_us[['branch', 'rating', 'location', 'year']].copy()
new_disney_df_us_cleaned

,branch,rating,location,year
10,Disneyland_HongKong,5,United States,2019
21,Disneyland_HongKong,3,United States,2019
22,Disneyland_HongKong,3,United States,2019
40,Disneyland_HongKong,3,United States,2019
43,Disneyland_HongKong,4,United States,2019
...,...,...,...,...
42082,Disneyland_Paris,5,United States,2011
42093,Disneyland_Paris,5,United States,2010
42100,Disneyland_Paris,1,United States,2011
42104,Disneyland_Paris,1,United States,2010


In [15]:
new_disney_df_us_cleaned_1 = new_disney_df_us_cleaned.loc[new_disney_df_us_cleaned['rating'] >=5]
new_disney_df_us_cleaned_1

,branch,rating,location,year
10,Disneyland_HongKong,5,United States,2019
51,Disneyland_HongKong,5,United States,2019
56,Disneyland_HongKong,5,United States,2019
62,Disneyland_HongKong,5,United States,2019
69,Disneyland_HongKong,5,United States,2019
...,...,...,...,...
41958,Disneyland_Paris,5,United States,2010
42013,Disneyland_Paris,5,United States,2011
42046,Disneyland_Paris,5,United States,2011
42082,Disneyland_Paris,5,United States,2011


In [16]:
new_disney_df_us_cleaned_1

,branch,rating,location,year
10,Disneyland_HongKong,5,United States,2019
51,Disneyland_HongKong,5,United States,2019
56,Disneyland_HongKong,5,United States,2019
62,Disneyland_HongKong,5,United States,2019
69,Disneyland_HongKong,5,United States,2019
...,...,...,...,...
41958,Disneyland_Paris,5,United States,2010
42013,Disneyland_Paris,5,United States,2011
42046,Disneyland_Paris,5,United States,2011
42082,Disneyland_Paris,5,United States,2011


In [25]:
rds_connection_string = "postgres:(pw)@localhost:5433/project2_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [26]:
cacao_us_df.to_sql(name='us_cacao_table', con=engine, if_exists='append', index=False)
new_disney_df_us_cleaned_1.to_sql(name='us_disney_table', con=engine, if_exists='append', index=False)

In [29]:
pd.read_sql_query('select * from us_cacao_table', con=engine).head()

,company,location,cacao_rating,bean_origin,year
0,Acalli,United States,3.75,Peru,2015
1,Acalli,United States,3.75,Peru,2015
2,Altus aka Cao Artisan,United States,2.75,Nicaragua,2016
3,Altus aka Cao Artisan,United States,3.00,Peru,2016
4,Altus aka Cao Artisan,United States,2.50,Bolivia,2016


In [28]:
pd.read_sql_query('select * from us_disney_table', con=engine).head()

,branch,rating,location,year
0,Disneyland_HongKong,5,United States,2019
1,Disneyland_HongKong,5,United States,2019
2,Disneyland_HongKong,5,United States,2019
3,Disneyland_HongKong,5,United States,2019
4,Disneyland_HongKong,5,United States,2019


In [30]:
disney_cacao_df = pd.read_sql_query('SELECT * FROM us_disney_table a inner join us_cacao_table e on a.year = cast(e.year as varchar(4))', con=engine)

In [31]:
disney_cacao_df = disney_cacao_df.drop_duplicates()
disney_cacao_df

,branch,rating,location,year,company,location,cacao_rating,bean_origin,year
0,Disneyland_HongKong,5,United States,2010,Snake & Butterfly,United States,3.0,Madagascar,2010
1,Disneyland_Paris,5,United States,2010,Snake & Butterfly,United States,3.0,Madagascar,2010
2,Disneyland_California,5,United States,2010,Snake & Butterfly,United States,3.0,Madagascar,2010
34,Disneyland_HongKong,5,United States,2010,Oakland Chocolate Co.,United States,2.5,Jamaica,2010
35,Disneyland_Paris,5,United States,2010,Oakland Chocolate Co.,United States,2.5,Jamaica,2010
...,...,...,...,...,...,...,...,...,...
718091,Disneyland_Paris,5,United States,2017,Letterpress,United States,3.5,Honduras,2017
718161,Disneyland_HongKong,5,United States,2017,Letterpress,United States,3.5,Honduras,2017
718231,Disneyland_California,5,United States,2017,Spencer,United States,3.5,Ecuador,2017
719004,Disneyland_Paris,5,United States,2017,Spencer,United States,3.5,Ecuador,2017


In [32]:
disney_cacao_df.to_sql(name='us_disney_cacao_table', con=engine, if_exists='append', index=False)